<a href="https://colab.research.google.com/github/tajibo/HomeworkDTS/blob/main/02_Tokens_and_Token_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Токены и вложения токенов</h1>
<i>Изучение токенов и вложений как неотъемлемой части построения LLM</i>

### [НЕОБЯЗАТЕЛЬНО]
Если вы просматриваете этот блокнот в Google Colab (или любом другом поставщике облачных услуг), вам нужно **раскомментировать и запустить** следующий блок кода, чтобы установить зависимости:

---

💡 **ПРИМЕЧАНИЕ**: для запуска примеров в этом блокноте нам понадобится графический процессор. В Google Colab перейдите в
**Среда выполнения > Изменить тип среды выполнения > Аппаратный ускоритель > Графический процессор > Тип графического процессора > T4**.

---

In [4]:
# %%capture
# !pip install transformers>=4.41.2 sentence-transformers>=3.0.1 gensim>=4.3.2 scikit-learn>=1.5.0 accelerate>=0.31.0

# Загрузка и запуск LLM

Первый шаг — загрузить нашу модель на GPU для более быстрого вывода. Обратите внимание, что мы загружаем модель и токенизатор отдельно и сохраняем их как таковые, чтобы мы могли исследовать их по отдельности.

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
!ip install transformers==4.41.2


Object "install" is unknown, try "ip help".


In [8]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cpu",
    torch_dtype="auto",
    trust_remote_code=True,
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)

# Загружаем модель с явным указанием attn_implementation
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    attn_implementation="eager"  # Явно указываем использование eager attention
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"

# Токенизируем входной промпт с явной передачей return_attention_mask=True
inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)
input_ids = inputs.input_ids.to("cuda")
attention_mask = inputs.attention_mask.to("cuda")

# Генерируем текст с передачей attention_mask
generation_output = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,  # Передаем attention_mask
    max_new_tokens=20
)

# Выводим результат
print(tokenizer.decode(generation_output[0], skip_special_tokens=True))


In [ ]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20
)

# Print the output
print(tokenizer.decode(generation_output[0]))

In [ ]:
print(input_ids)

In [ ]:
for id in input_ids[0]:
   print(tokenizer.decode(id))

In [ ]:
generation_output

In [ ]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

# Сравнение обученных токенизаторов LLM

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

In [ ]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

In [ ]:
show_tokens(text, "bert-base-uncased")

In [ ]:
show_tokens(text, "bert-base-cased")

In [ ]:
show_tokens(text, "gpt2")

In [ ]:
show_tokens(text, "google/flan-t5-small")

In [ ]:
# The official is `tiktoken` but this the same tokenizer on the HF platform
show_tokens(text, "Xenova/gpt-4")

In [ ]:
# You need to request access before being able to use this tokenizer
show_tokens(text, "bigcode/starcoder2-15b")

In [ ]:
show_tokens(text, "facebook/galactica-1.3b")

In [ ]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

# Контекстуализированные эмбединги слов из языковой модели (например, BERT)

In [ ]:
from transformers import AutoModel, AutoTokenizer

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

# Load a language model
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# Tokenize the sentence
tokens = tokenizer('Hello world', return_tensors='pt')

# Process the tokens
output = model(**tokens)[0]

In [ ]:
output.shape

In [ ]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

In [ ]:
output

# Встраивание текста (для предложений и целых документов)

In [ ]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Convert text to text embeddings
vector = model.encode("Best movie ever!")

In [ ]:
vector.shape

# Word Embeddings до LLMs


In [ ]:
!pip install gensim

In [ ]:
import gensim.downloader as api

# Download embeddings (66MB, glove, trained on wikipedia, vector size: 50)
# Other options include "word2vec-google-news-300"
# More options at https://github.com/RaRe-Technologies/gensim-data
model = api.load("glove-wiki-gigaword-50")

In [ ]:
model.most_similar([model['king']], topn=11)

# # Рекомендация песен по embeddings

In [ ]:
import pandas as pd
from urllib import request

# Get the playlist dataset file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# Parse the playlist dataset file. Skip the first two lines as
# they only contain metadata
lines = data.read().decode("utf-8").split('\n')[2:]

# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# Load song metadata
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [ ]:
print( 'Playlist #1:\n ', playlists[0], '\n')
print( 'Playlist #2:\n ', playlists[1])

In [ ]:
from gensim.models import Word2Vec

# Train our Word2Vec model
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

In [ ]:
song_id = 2172

# Ask the model for songs similar to song #2172
model.wv.most_similar(positive=str(song_id))

In [ ]:
print(songs_df.iloc[2172])

***Допишите функцию рекомендации похожих песен***

In [ ]:
import numpy as np

def print_recommendations(song_id, topn=5):
    print(f"Рекомендации для песни {song_id}:")
    try:
        # Получаем topn похожих песен по строковому представлению song_id
        recommendations = model.wv.most_similar(positive=[str(song_id)], topn=topn)
    except KeyError:
        print(f"Песня с id {song_id} не найдена в модельном пространстве.")
        return

    for rec_id, similarity in recommendations:
        try:
            # Извлекаем информацию о песне по идентификатору rec_id
            song_info = songs_df.loc[rec_id]
            title = song_info['title']
            artist = song_info['artist']
            print(f"  {title} - {artist} (схожесть: {similarity:.2f})")
        except KeyError:
            print(f"  Песня с id {rec_id} не найдена в базе данных.")


In [ ]:
print_recommendations(2172)

In [ ]:
print_recommendations(842)